# tf.data performance tips
Distributed training model, the efficiency with which load data can often become critical and While creating dataset make it as global batch size.

**Calling dataset.cache()**
when using .cache() data will cached after runnig through the first iteration over the data, each subsequent iteration will use the cached data

These method can improve the preformance when data is not expected to change from iteration to iteration,reading data from a remote distributed filesystem, reading data from local disk, but data would fit in memory and workflow is significantly IO-bound.


**Calling dataset.prefetch(buffer_size)**

call .prefetch(buffer_size) after creating a dataset,data pipeline will run asynchronously from your model, with new samples being preprocessed and stored in a buffer while the current batch samples are used to train the model. The next batch will be prefetched in GPU memory by the time the current batch is over.



# Multi-worker distributed synchronous training.
This setup multiple machine each with one or several GPUs on them. Single-host training, each available GPU will run one model replica and the value of the variable of each replica is kept in sync after each batch.The current implementation assumes that all workers have the same number of GPUs

Uses:

1. Setup a cluster.
2. Setup an appropriate TF_CONFIG enviroment variable on each worker.
3. Run model construction and compilation code within the scope of a MultiWokerMirroredStrategy.obejct.
4. Run evaluation code on a designated evalutor machine.

Setting up cluster

Setup a cluster.Each machine individually  should be setup  so as to be able to run model and to able to access data source.

Cluster management is beyond the scope

**Setting up the `TF_CONFIG` environment variable**

one significant difference between the single-host workflow and the multi-worker workflow is that you need to set a `TF_CONFIG` environment variable on each machine running in cluster.

The `TF_CONFIG` environment variable is a JSON string that specifies:

- The cluster configuration, while the list of addresses & ports of the machines that
make up the cluster
- The worker's "task", which is the role that this specific machine has to play within
the cluster.

One example of TF_CONFIG is:

```
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
```

The multi-worker synchronous training setup, valid roles (task types) for the machines
are "worker" and "evaluator".

- The workers train the model, each one processing sub-batches of a global batch.

- One of the workers will serve as "chief", a particular kind of worker that
is responsible for saving logs and checkpoints for later reuse 

- The evaluator runs a continuous loop that loads the latest checkpoint saved by the
chief worker, runs evaluation on it 


**Running code on each worker**

Run training code on each worker and evaluation code on the evaluator.

The training code is basically the same as the single-host setup,
except using `MultiWorkerMirroredStrategy` instead of `MirroredStrategy`.


The evaluator would simply use `MirroredStrategy` and `model.evaluate()`. This will loading the latest checkpoint saved by the chief worker to a Cloud storage location, and
would save evaluation logs to the same location as the chief logs.


Example: Multi-worker setup
```
# Set TF_CONFIG
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
 
 
# Open a strategy scope and create/restore the model.
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
with strategy.scope():
  model = make_or_restore_model()
 
callbacks = [
    # This callback saves a SavedModel every 100 batches
    keras.callbacks.ModelCheckpoint(filepath='path/to/cloud/location/ckpt',
                                    save_freq=100),
    keras.callbacks.TensorBoard('path/to/cloud/location/tb/')
]
model.fit(train_dataset,
          callbacks=callbacks,
          ...)
On other workers:

# Set TF_CONFIG
worker_index = 1  # For instance
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': worker_index}
})
 ```
 
# Open a strategy scope and create/restore the model.
# You can restore from the checkpoint saved by the chief.

```
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
with strategy.scope():
  model = make_or_restore_model()
 
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath='local/path/ckpt', save_freq=100),
    keras.callbacks.TensorBoard('local/path/tb/')
]
model.fit(train_dataset,
          callbacks=callbacks,
          ...)
On the evaluator:

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model = make_or_restore_model()  # Restore from the checkpoint saved by the chief.
 
results = model.evaluate(val_dataset)
# Then, log the results on a shared location, write TensorBoard logs, etc
```